In [1]:
import librosa
import librosa.display
import librosa.feature
import tensorflow.compat.v1 as tf


#from scripts
from Attention import *
from arguments import parse_arguments 
import torch
import numpy as np
from utils import *
from Attention import *
from data_utils import *
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler


2023-10-05 18:33:19.911750: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-05 18:33:19.912938: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 18:33:19.936711: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 18:33:19.937314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-05 18:33:20.303536: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
from vggish_smoke_test import *
# import vggish_slim
# import vggish_params
# import vggish_input


Testing your install of VGGish

Resampling via resampy works!
Log Mel Spectrogram example:  [[-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 ...
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]]
INFO:tensorflow:Restoring parameters from vggish_model.ckpt


2023-10-05 18:33:22.080910: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-05 18:33:22.081256: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/kary/anaconda3/envs/musical/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.

VGGish embedding:  [-2.72986382e-01 -1.80314243e-01  5.19921482e-02 -1.43571466e-01
 -1.04673848e-01 -4.96598154e-01 -1.75267860e-01  4.23147976e-01
 -8.22126091e-01 -2.16801703e-01 -1.17509484e-01 -6.70077085e-01
  1.43174529e-01 -1.44183964e-01  8.73494893e-03 -8.71973336e-02
 -1.84393510e-01  5.96655548e-01 -3.43809605e-01 -5.79106510e-02
 -1.65071234e-01  4.22911346e-02 -2.55293489e-01 -2.36356825e-01
  1.80295452e-01  3.02612156e-01  1.08356804e-01 -4.48397934e-01
  1.22757599e-01 -2.99955249e-01 -5.55934012e-01  5.05966663e-01
  2.05210567e-01  8.87592018e-01  9.03702617e-01 -2.10566297e-01
 -3.27461362e-02  1.38691485e-01 -2.27416620e-01  1.14804089e-01
  5.95409870e-01 -4.76971269e-01  2.28232607e-01  1.54626817e-01
  1.64934054e-01  7.19253302e-01  1.24101830e+00  5.61996639e-01
  2.73531914e-01  3.09790373e-02  2.10977703e-01 -6.09551966e-01
 -3.15282673e-01  1.76392615e-01 -8.96194577e-02 -4.26822454e-01
  3.12993884e-01 -1.56592414e-01  3.31673682e-01  1.29436329e-01
  1.66

In [3]:
def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
  """Define VGGish model, load the checkpoint, and return a dictionary that points
  to the different tensors defined by the model.
  """
  vggish_slim.define_vggish_slim()
  checkpoint_path = 'vggish_model.ckpt'
  vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)
  features_tensor = sess.graph.get_tensor_by_name(
      vggish_params.INPUT_TENSOR_NAME)
  embedding_tensor = sess.graph.get_tensor_by_name(
      vggish_params.OUTPUT_TENSOR_NAME)
  layers = {'conv1': 'vggish/conv1/Relu',
            'pool1': 'vggish/pool1/MaxPool',
            'conv2': 'vggish/conv2/Relu',
            'pool2': 'vggish/pool2/MaxPool',
            'conv3': 'vggish/conv3/conv3_2/Relu',
            'pool3': 'vggish/pool3/MaxPool',
            'conv4': 'vggish/conv4/conv4_2/Relu',
            'pool4': 'vggish/pool4/MaxPool',
            'fc1': 'vggish/fc1/fc1_2/Relu',
            #'fc2': 'vggish/fc2/Relu',
            'embedding': 'vggish/embedding',
            'features': 'vggish/input_features',
         }
  g = tf.get_default_graph()
  for k in layers:
    layers[k] = g.get_tensor_by_name( layers[k] + ':0')
  return {'features': features_tensor,
          'embedding': embedding_tensor,
          'layers': layers,
         }

In [4]:
def _ProcessWithVGGish(vgg, x, sr):
  '''Run the VGGish model, starting with a sound (x) at sample rate
  (sr). Return a whitened version of the embeddings. Sound must be scaled to be
  floats between -1 and +1.'''
  # Produce a batch of log mel spectrogram examples.
  input_batch = vggish_input.waveform_to_examples(x, sr)
  # print('Log Mel Spectrogram example: ', input_batch[0])
  [embedding_batch] = sess.run([vgg['embedding']],
                               feed_dict={vgg['features']: input_batch})
  # Postprocess the results to produce whitened quantized embeddings.
  pca_params_path = 'vggish_pca_params.npz'
  pproc = vggish_postprocess.Postprocessor(pca_params_path)
  postprocessed_batch = pproc.postprocess(embedding_batch)
  # print('Postprocessed VGGish embedding: ', postprocessed_batch[0])
  return postprocessed_batch


In [5]:
tf.compat.v1.disable_eager_execution()
tf.reset_default_graph()
sess = tf.Session()
vgg = CreateVGGishNetwork(0.21)

INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [6]:
y_v, sr_v  = librosa.load('Demon-Slayer.mp3')

In [7]:
X_v = _ProcessWithVGGish(vgg, y_v, sr_v)

In [8]:
k_v1, k_v2 = X_v.shape
X_v = np.asarray(X_v).reshape(-1,k_v1,k_v2)
X_tst_torch_v = (torch.from_numpy(X_v)).type(torch.float32)
X_tst_torch_v =(X_tst_torch_v/255)

In [9]:
device = torch.device('cpu') #'cuda' if torch.cuda.is_available() else 

In [10]:
modelPath = 'log/Attention/Attention/trainedModel_0.pth'

model = DecisionLevelSingleAttention(
                freq_bins=128,
                classes_num=20,
                emb_layers=3,
                hidden_units=128,
                drop_rate=0.6)


#Restore model
model.load_state_dict(torch.load(modelPath))
model = model.eval()
model = model.to(device)


with torch.no_grad():
    output_v = model(X_tst_torch_v)

output_v = to_numpy(output_v).ravel()
# print(output_v.shape)
# print(output_v) 
outputIndex = np.where(output_v >= 0.9)
# print(outputIndex)


55336


In [11]:
classMap= {'accordion': 0,
 'banjo': 1,
 'bass': 2,
 'cello': 3,
 'clarinet': 4,
 'cymbals': 5,
 'drums': 6,
 'flute': 7,
 'guitar': 8,
 'mallet_percussion': 9,
 'mandolin': 10,
 'organ': 11,
 'piano': 12,
 'saxophone': 13,
 'synthesizer': 14,
 'trombone': 15,
 'trumpet': 16,
 'ukulele': 17,
 'violin': 18,
 'voice': 19}
inv_map = {v: k for k, v in classMap.items()}


In [12]:
print("List of detected musical instruments: ")
for j,i in enumerate(outputIndex[0]):
    print(str(j+1)+". "+inv_map[i])
    

List of detected musical instruments: 
1. saxophone
2. trombone
3. trumpet
4. voice


In [13]:
# accordion 0.06983575
# banjo 0.33101374
# bass 0.020017007
# cello 0.06161041
# clarinet 0.12218426
# cymbals 0.0016985144
# drums 0.014519502
# flute 0.700888
# guitar 0.8985557
# mallet_percussion 0.5362431
# mandolin 0.59339917
# organ 0.045906793
# piano 0.7231139
# saxophone 0.0318367
# synthesizer 0.23019123
# trombone 0.010454232
# trumpet 0.025121506
# ukulele 0.4342751
# violin 0.25603348
# voice 0.0148326345

# flute, guitar,mallet_percussion, piano, 